### Some markdown is never a bad idea


In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

import seaborn as sns
import itertools
# Tools for machine learning
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline

In [3]:
matches = pd.read_csv('../data/seasons_merged.csv')
letter_to_result = {'H': 1, 'D': 0, 'A': -1}

matches

,Unnamed: 0,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HY,AY,HR,AR,B365H,B365D,B365A,BbAvH,BbAvD,BbAvA
0,0,2009-08-15,Aston Villa,Wigan,0,2,-1,0,1,A,...,2,2,0,0,1.67,3.60,5.50,1.66,3.51,5.33
1,1,2009-08-15,Blackburn,Man City,0,2,-1,0,1,A,...,2,1,0,0,3.60,3.25,2.10,3.37,3.24,2.12
2,2,2009-08-15,Bolton,Sunderland,0,1,-1,0,1,A,...,2,1,0,0,2.25,3.25,3.25,2.24,3.20,3.15
3,3,2009-08-15,Chelsea,Hull,2,1,1,1,1,D,...,1,2,0,0,1.17,6.50,21.00,1.17,6.26,16.39
4,4,2009-08-15,Everton,Arsenal,1,6,-1,0,3,A,...,0,0,0,0,3.20,3.25,2.30,3.07,3.21,2.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,3795,2019-12-05,Liverpool,Wolves,2,0,1,1,0,H,...,0,2,0,0,1.30,6.00,11.00,1.30,5.62,10.17
3796,3796,2019-12-05,Man United,Cardiff,0,2,-1,0,1,A,...,3,3,0,0,1.28,6.50,11.00,1.28,6.18,10.10
3797,3797,2019-12-05,Southampton,Huddersfield,1,1,0,1,0,H,...,0,1,0,0,1.44,4.75,8.50,1.42,4.81,7.64
3798,3798,2019-12-05,Tottenham,Everton,2,2,0,1,0,H,...,0,2,0,0,2.20,3.50,3.50,2.09,3.51,3.58


In [4]:
def get_n_last_matches(matches, date, team, n = 10):
    '''
    Get the last n matches of a given team.
    '''
    # All matches with a given team
    team_matches = matches[(matches['HomeTeam'] == team) | (matches['AwayTeam'] == team)]
    
    #Filter n last matches from team matches
    n_last_matches = (team_matches[team_matches.Date < date]
                      .sort_values(by = 'Date', ascending = False)
                      .iloc[0:n,:])
    
    return n_last_matches


def get_n_last_matches_against_each_other(matches, date, home_team, away_team, n = 5):
    ''' 
    Get the last n matches of two given teams. If possible, else
    get all matches available
    '''
    
    home_matches = matches[(matches['HomeTeam'] == home_team) & (matches['AwayTeam'] == away_team)]    
    away_matches = matches[(matches['HomeTeam'] == away_team) & (matches['AwayTeam'] == home_team)]  
    total_matches = pd.concat([home_matches, away_matches])
    
    # Get last n matches, if possible:
    try:    
        last_matches = (total_matches[total_matches.Date < date]
                        .sort_values(by = 'Date', ascending = False)
                        .iloc[0:n,:])
    except:  # If there are less than n matches
        last_matches = (total_matches[total_matches.Date < date]
                        .sort_values(by = 'Date', ascending = False)
                        .iloc[0:total_matches.shape[0],:])

    return last_matches


def get_goals(matches, team):
    '''
    Get total number of goals,a specfic team has scored 
    from a dataframe of specific matches
    '''
    home_goals = matches.FTHG[matches.HomeTeam == team].sum()
    away_goals = matches.FTAG[matches.AwayTeam == team].sum()

    return home_goals + away_goals

def get_concealed_goals(matches, team):
    '''
    Get all the goals, concealed of a specfic team from a dataframe of specific matches
    '''
    home_goals = matches.FTAG[matches.HomeTeam == team].sum()
    away_goals = matches.FTHG[matches.AwayTeam == team].sum()

    return home_goals + away_goals

def get_wins(matches, team):
    '''
    Get the number of wins of a specfic team from a dataframe of specific matches.
    '''
    home_wins = matches[(matches.FTR == 1) & (matches.HomeTeam == team)].shape[0]
    away_wins = matches[(matches.FTR == -1) & (matches.AwayTeam == team)].shape[0]

    return home_wins + away_wins


def coefficients_to_probability(matches):
    '''
    Converts betting platform coefficient(1 < x) with % of income
    into a probability coefficient(0 < x < 1)
    '''
    # How many profit betting companies make on bets
    matches['profit_B365'] = sum((1/matches['B365H'], 1/matches['B365D'], 1/matches['B365A']))
    matches['profit_BbAv'] = sum((1/matches['BbAvA'], 1/matches['BbAvD'], 1/matches['BbAvH']))

    # Converting all betting coefficients into probabilities of homw/draw/away:
    for betting_column in ['B365H', 'B365D', 'B365A', 'BbAvH', 'BbAvD', 'BbAvA']:
        matches[betting_column] = 1 / (matches[betting_column] * matches['profit_' + betting_column[:-1]])

    return matches
# get_n_last_matches_against_each_other(matches, '2020-12-15', 'Liverpool', 'Tottenham', 10)


In [5]:
# Create features, based on which, the model would train and predict results

def get_features_for_match(match, matches, n1=10, n2=3):
    '''
    Creates a special set of features for each match, if possible(10 last matches
    and 3 last matches against each other)
    '''
    match_date = match.Date
    home_team = match.HomeTeam
    away_team = match.AwayTeam
    # Get n1 last matches of 2 teams
    home_last = get_n_last_matches(matches, match_date, home_team, n=n1)
    away_last = get_n_last_matches(matches, match_date, away_team, n=n1)
    # Get last n2 matches against each other
    home_last_against = get_n_last_matches_against_each_other(matches, match_date, home_team, away_team, n=n2)
    away_last_against = get_n_last_matches_against_each_other(matches, match_date, away_team, home_team, n=n2)
    # Goals stuff
    home_goals = get_goals(home_last, home_team)
    away_goals = get_goals(away_last, away_team)
    home_goals_conceided = get_concealed_goals(home_last, home_team)
    away_goals_conceided = get_concealed_goals(away_last, away_team)

    res = pd.DataFrame()
    res.loc[0, 'H_goal_diff'] = home_goals - home_goals_conceided
    res.loc[0, 'A_goal_diff'] = away_goals - away_goals_conceided
    res.loc[0, 'H_win'] = get_wins(home_last, home_team) 
    res.loc[0, 'A_win'] = get_wins(away_last, away_team)
    res.loc[0, 'H_win_against'] = get_wins(home_last_against, home_team)
    res.loc[0, 'A_win_against'] = get_wins(away_last_against, away_team)
    # TODO ПОПРООБУВАТИ ЩЕ ЯКІСЬ КРИТЕРІЇ ПОТЕСТУВАТИ
#     print(result.loc[0])
    return res.loc[0]


def create_features(matches):
    '''
    Iterate throu all matches, create features for every single of them
    if possible and aggregate them together
    '''
    print('Generating features... Please wait for one or two minutes')
    # Creates dataframe with features for all matches
    matches_features = matches.apply(lambda x: get_features_for_match(x, matches, n1=12, n2=3), axis=1)
    return matches_features

def explore_features(features, inputs, path):
    '''
    Explore data by plotting KDE graphs.
    '''
    fig = plt.figure(1)
    fig.subplots_adjust(bottom= -1, left=0.025, top = 2, right=0.975)
    #Loop through features    
    i = 1
    for col in features.columns:
        sns.set_style("whitegrid")
        sns.set_context("paper", font_scale = 0.5, rc={"lines.linewidth": 1})
        plt.subplot(5,5,0 + i)
        j = i - 1
        #Plot KDE for all labels
        try:
            sns.distplot(inputs[inputs['label'] == 'H'].iloc[:,j], hist = False, label = 'Home')
            sns.distplot(inputs[inputs['label'] == 'D'].iloc[:,j], hist = False, label = 'Draw')
            sns.distplot(inputs[inputs['label'] == 'A'].iloc[:,j], hist = False, label = 'Away')
            plt.legend();
        except ValueError as e:
            print(e)
        i = i + 1
    #Define plot format    
    DefaultSize = fig.get_size_inches()
    fig.set_size_inches((DefaultSize[0]*1.2, DefaultSize[1]*1.2))
    plt.show()
    #Compute and print label weights
    labels = inputs.loc[:,'label']
    class_weights = labels.value_counts() / len(labels)
    print(class_weights)
    #Store description of all features
    feature_details = features.describe().transpose()
    #Return feature details
    return feature_details

In [6]:
# Getting features ready

time_start = time.time()
features = create_features(matches)
features2 = features.copy()
print('Generated features in', time.time() - time_start, 'sec.')




Generating features... Please wait for one or two minutes
Generated features in 260.20324540138245 sec.


In [77]:
inputs = features2.copy()
labels = matches.loc[:, 'HTR']
labels.name = 'label'

# # Printing statistics to see the correlations:
# statistic = explore_features(features, inputs, '/soccer/')
# statistic

# features = coefficients_to_probability(matches)[['B365H','B365D','B365A','BbAvH','BbAvD','BbAvA']]
# features = pd.merge(features, inputs, left_index=True, right_index=True)
features = inputs
features

,H_goal_diff,A_goal_diff,H_win,A_win,H_win_against,A_win_against
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
3795,20.0,2.0,10.0,5.0,3.0,0.0
3796,-1.0,-11.0,5.0,3.0,2.0,0.0
3797,-3.0,-20.0,5.0,1.0,1.0,0.0
3798,1.0,8.0,4.0,6.0,3.0,0.0


# Machine learning part 


In [78]:
# dependencies
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
from sklearn.linear_model import LogisticRegression
#A random forest is a meta estimator that fits a number of decision tree classifiers 
#on various sub-samples of the dataset and use averaging to improve the predictive 
#accuracy and control over-fitting.
from sklearn.ensemble import RandomForestClassifier
#a discriminative classifier formally defined by a separating hyperplane.
from sklearn.svm import SVC
#displayd data
from IPython.display import display

In [79]:
#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict_proba(features)
    
    end = time()
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    # f1_score(target, y_pred, pos_label='H'),
    return sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)

    # Print the results of prediction for both training and testing
    #f1, 
    f1 = 'hz'
    acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    #f1, 
    acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

In [80]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, 
                                                    test_size = 50,
                                                    random_state = 2,
                                                    stratify = labels)
X_test.head()

,H_goal_diff,A_goal_diff,H_win,A_win,H_win_against,A_win_against
1496,-8.0,-5.0,2.0,2.0,2.0,0.0
2214,4.0,-10.0,4.0,1.0,3.0,0.0
2225,15.0,-5.0,7.0,3.0,2.0,1.0
1290,-14.0,-6.0,1.0,2.0,0.0,1.0
1263,1.0,-7.0,4.0,4.0,0.0,2.0


In [110]:
# Splitting the data into training and test sets:
X_train, X_test, y_train, y_test = train_test_split(features, labels, 
                                                    test_size = 50,
                                                    random_state = 2,
                                                    stratify = labels)

# Classifiers
y_train = y_train.apply(lambda x: letter_to_result[x])

clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 912, kernel='rbf')
clf_C = xgb.XGBClassifier(seed = 82)

# y_train = y_train.apply(lambda x: 1 if x == 1 else 0)
# clf_C.fit(X_train, y_train)
# train_predict(clf_A, X_train, y_train, X_test, y_test)
# print()
# train_predict(clf_B, X_train, y_train, X_test, y_test)
# print()
# train_predict(clf_C, X_train, y_train, X_test, y_test)
# print()

clf_C.fit(X_train, y_train)


# make a prediction
ynew = clf_C.predict_proba(X_test.iloc[-1:])

print(ynew[0][0], type(ynew))
X_test.iloc[-1:]

0.09773014 <class 'numpy.ndarray'>


,H_goal_diff,A_goal_diff,H_win,A_win,H_win_against,A_win_against
3322,19.0,-7.0,9.0,2.0,0.0,1.0


### Методом спроб і помилок я визначив, що під цю проблему найкраще підходить XGBClassifier. 
Тому лишилось його відкалібрувати

In [ ]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

parameters = { 'learning_rate' : [0.1],
               'n_estimators' : [40],
               'max_depth': [3],
               'min_child_weight': [3],
               'gamma':[0.4],
               'subsample' : [0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             }  

# TODO: Initialize the classifier
clf = xgb.XGBClassifier(seed=2)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='H')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf

# Report the final F1 score for training and testing after parameter tuning
f1, acc = predict_labels(clf, X_train, y_train)
print "F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc)
    
f1, acc = predict_labels(clf, X_test, y_test)
print "F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.4, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=3, missing=None, n_estimators=40, nthread=-1,
       objective='binary:logistic', reg_alpha=1e-05, reg_lambda=1,
       scale_pos_weight=1, seed=2, silent=True, subsample=0.8)
Made predictions in 0.0150 seconds.
F1 score and accuracy score for training set: 0.6365 , 0.6827.
Made predictions in 0.0000 seconds.
F1 score and accuracy score for test set: 0.7826 , 0.8000.

